In [ ]:
import os
import sys

from matplotlib import pyplot as plt
import matplotlib.patches as mpatch
import numpy as np
from scipy.integrate import dblquad
from scipy.stats import multivariate_normal

CWD = os.path.abspath("")
sys.path.append(os.path.join(CWD, ".."))
from plt_settings import plt_settings

full_width = 5.5
ratio = 1 / 1.618

In [ ]:
ITERATIONS = int(5e6)
BURN_IN = int(4e6)

sampling_data = np.load(os.path.join(CWD, "2d_gaussian.npz"))

projlmc_samples_all = sampling_data["projlmc_x"]
projlmc_samples = projlmc_samples_all[BURN_IN:]
pdlmc_samples_all = sampling_data["pdlmc_x"]
pdlmc_samples = pdlmc_samples_all[BURN_IN:]
pdlmc_lambda = sampling_data["pdlmc_lambda"]
pdlmc_nu = sampling_data["pdlmc_nu"]
rejection_samples = sampling_data["rejection_x"]

sampling_data = np.load(os.path.join(CWD, "mirror-langevin/scripts/2d_mirror.npz"))
mirror_samples = sampling_data["mirror_x"][BURN_IN:]

In [ ]:
# True mean
Z, _ = dblquad(
    lambda y, x: multivariate_normal.pdf([x, y], mean=np.ones(2) * 2),
    -1,
    1,
    lambda x: -np.sqrt(1 - x**2),
    lambda x: np.sqrt(1 - x**2),
)
true_mean_x, _ = dblquad(
    lambda y, x: x * multivariate_normal.pdf([x, y], mean=np.ones(2) * 2),
    -1,
    1,
    lambda x: -np.sqrt(1 - x**2),
    lambda x: np.sqrt(1 - x**2),
)

true_mean = np.array([true_mean_x / Z, true_mean_x / Z])
print(f"Mean of the distribution: {true_mean}")

# Estimated mean
pdlmc_mean = pdlmc_samples.mean(axis=0)
projlmc_mean = projlmc_samples.mean(axis=0)
rejection_mean = rejection_samples.mean(axis=0)
mirror_mean = mirror_samples.mean(axis=0)

print(f"Estimated mean via PD-LMC: {pdlmc_mean}")
print(f"Estimated mean via Proj-LMC: {projlmc_mean}")
print(f"Estimated mean via rejection: {rejection_mean}")
print(f"Estimated mean via Mirror LMC: {mirror_mean}")

# Out-of-support samples
out_of_support = ((pdlmc_samples * pdlmc_samples).sum(axis=1) > 1).mean()
print(f"Percentage of out-of-support samples: {out_of_support * 100:.2f}%")

# overestimation of boundary
pdlmc_samples_norm = (pdlmc_samples * pdlmc_samples).sum(axis=1)
pdlmc_boundary = np.logical_and(pdlmc_samples_norm >= 1 - 1e-3, pdlmc_samples_norm <= 1).mean()
projlmc_boundary = ((projlmc_samples * projlmc_samples).sum(axis=1) >= 1 - 1e-3).mean()
rejection_boundary = ((rejection_samples * rejection_samples).sum(axis=1) >= 1 - 1e-3).mean()
mirror_boundary = ((mirror_samples * mirror_samples).sum(axis=1) >= 1 - 1e-3).mean()

print(f"PD-LMC - Percentage of samples on boundary: {pdlmc_boundary * 100:.2f}%")
print(f"Proj-LMC - Percentage of samples on boundary: {projlmc_boundary * 100:.2f}%")
print(f"Rejection - Percentage of samples on boundary: {rejection_boundary * 100:.2f}%")
print(f"Mirror LMC - Percentage of samples on boundary: {mirror_boundary * 100:.2f}%")

In [ ]:
plt_settings["figure.figsize"] = (full_width, full_width / 4)
scatter_kwargs = {
    "s": 20.0,
    "edgecolors": "face",
    "alpha": 0.03,
    "linewidth": 0.0,
}

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 4, dpi=300)
    axs[0].scatter(pdlmc_samples[::200, 0], pdlmc_samples[::200, 1], **scatter_kwargs)
    axs[1].scatter(projlmc_samples[::200, 0], projlmc_samples[::200, 1], **scatter_kwargs)
    axs[2].scatter(mirror_samples[::200, 0], mirror_samples[::200, 1], **scatter_kwargs)
    axs[3].scatter(
        rejection_samples[::200, 0], rejection_samples[::200, 1], c="grey", **scatter_kwargs
    )

    for i, m in enumerate([pdlmc_mean, projlmc_mean, mirror_mean[0.001], rejection_mean]):
        axs[i].scatter([m[0]], [m[1]], marker="x", color="C1", label="Estimated mean")
        axs[i].scatter([true_mean[0]], [true_mean[1]], marker="+", color="C3", label="True mean")

    # Circle
    for ax in axs:
        ax.add_artist(mpatch.Circle((0.0, 0.0), 1.0, facecolor="none", edgecolor="k", lw=0.7))
        ax.set_xlim((-1.3, 1.3))
        ax.set_ylim((-1.3, 1.3))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect(1)
        ax.grid()

    # Title
    for i, label in enumerate(["PD-LMC", "Proj. LMC", "Mirror LMC", "Rejection"]):
        axs[i].set_title(label, loc="right", x=0.97, y=0.02, pad=0)

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    axs.plot(pdlmc_lambda)
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_ylabel(r"Dual variable ($\lambda$)")

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, ratio * full_width / 2)

with plt.rc_context(plt_settings):
    _, axs = plt.subplots(1, 1, dpi=300)
    cum_mean = np.cumsum(
        np.clip(np.sum(pdlmc_samples_all * pdlmc_samples_all, axis=1) - 1.0, 0.0, None) - 0.001
    ) / np.arange(1, len(pdlmc_samples_all) + 1)
    axs.plot(cum_mean)
    axs.set_xlim((-1e3, 1e5))
    axs.grid()
    axs.set_xlabel("Iteration")
    axs.set_ylabel(r"Ergodic constraint slack")
    axs.ticklabel_format(scilimits=(-2, 2))

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, full_width / 2)

with plt.rc_context(plt_settings):
    fig, axs = plt.subplots(2, 2, dpi=300)
    iters = [10_000, 100_000, 1_000_000, 5_000_000]
    iters_label = ["1e4", "1e5", "1e6", "5e6"]
    for idx, ax in enumerate(axs.reshape(-1)):
        samples = projlmc_samples_all[: iters[idx]]
        ax.hexbin(
            samples[:, 0],
            samples[:, 1],
            gridsize=50,
            cmap="BuPu",
            extent=(-2, 2, -2, 2),
        )
        ax.add_artist(mpatch.Circle((0.0, 0.0), 1.0, facecolor="none", edgecolor="k", lw=0.5))
        ax.set(xlim=(-0.1, 1.1), ylim=(-0.1, 1.1))
        ax.set_title(f"Iter. = {iters_label[idx]}", loc="left", x=0.03, y=0.03, pad=0)
        ax.set_aspect(1.0)
        ax.set_xticks([])
        ax.set_yticks([])
    fig.suptitle("Proj. LMC")

    plt.show()

In [ ]:
plt_settings["figure.figsize"] = (full_width / 2, full_width / 2)

with plt.rc_context(plt_settings):
    fig, axs = plt.subplots(2, 2, dpi=300)
    iters = [10_000, 100_000, 1_000_000, 5_000_000]
    iters_label = ["1e4", "1e5", "1e6", "5e6"]
    for idx, ax in enumerate(axs.reshape(-1)):
        samples = pdlmc_samples_all[: iters[idx]]
        ax.hexbin(
            samples[:, 0],
            samples[:, 1],
            gridsize=50,
            cmap="BuPu",
            extent=(-2, 2, -2, 2),
        )
        ax.add_artist(mpatch.Circle((0.0, 0.0), 1.0, facecolor="none", edgecolor="k", lw=0.5))
        ax.set(xlim=(-0.1, 1.1), ylim=(-0.1, 1.1))
        ax.set_title(f"Iter. = {iters_label[idx]}", loc="left", x=0.03, y=0.03, pad=0)
        ax.set_aspect(1.0)
        ax.set_xticks([])
        ax.set_yticks([])
    fig.suptitle("PD-LMC")

    plt.show()